# PyChe Tutorial: Run + Diagnostic Plots

This notebook shows:
- in-memory runs (`return_results=True`)
- diagnostics computed directly from `res.mod` and `res.fis`
- loading a saved run back into Python
- generating diagnostic plots
- optional Cython compile commands


In [ ]:
from pyche import GCEModel, create_diagnostic_plots, read_outputs
from pyche.diagnostics import diagnostics_from_tables
import matplotlib.pyplot as plt


## 1) In-memory run


In [ ]:
m = GCEModel()
res = m.MinGCE(
    endoftime=500,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=False,
    backend='auto',
    output_mode='dataframe',
    write_output=False,
    return_results=True,
)
res.mod.shape, res.fis.shape


## 2) Diagnostics from `res.mod` and `res.fis`


In [ ]:
diag = diagnostics_from_tables(res.mod, res.fis)
diag


In [ ]:
fis_cols = {name: i for i, name in enumerate(res.fis_columns)}
t = res.fis[:, fis_cols['time']]
sfr = res.fis[:, fis_cols['sfr']]
zeta = res.fis[:, fis_cols['zeta']]

fig, ax = plt.subplots(1, 2, figsize=(10, 4))
ax[0].plot(t, sfr)
ax[0].set_title('SFR vs Time')
ax[0].set_xlabel('Time')
ax[1].plot(t, zeta)
ax[1].set_title('Zeta vs Time')
ax[1].set_xlabel('Time')
plt.tight_layout()


## 3) Save a run, load it, then build diagnostic plot files


In [ ]:
out_dir = 'RISULTATI_PYCHE_NOTEBOOK'
m.MinGCE(
    endoftime=1000,
    sigmat=3000.0,
    sigmah=50.0,
    psfr=0.3,
    pwind=0.0,
    delay=10000,
    time_wind=10000,
    use_mpi=False,
    show_progress=False,
    output_dir=out_dir,
    output_mode='dataframe',
    df_binary_format='pickle',
    write_output=True,
    return_results=False,
)
payload = read_outputs(out_dir, prefer='dataframe', binary_format='pickle')
payload['mod'].shape, payload['fis'].shape, payload['format']


In [ ]:
paths = create_diagnostic_plots(out_dir, prefer='dataframe', binary_format='pickle')
paths


## 4) Optional Cython compile commands
Run these in a shell from repository root before using `backend='cython'`:

```bash
pip install cython
python setup.py build_ext --inplace
```
